#shit

In [ ]:
!pip install gwpy
import gwpy
from gwpy.timeseries import TimeSeries

In [ ]:
!pip install librosa

In [ ]:
!pip install nnAudio

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c g2net-gravitational-wave-detection

In [ ]:
!mkdir tfm_g2n

In [ ]:
!unzip g2net-gravitational-wave-detection.zip -d tfm_g2n

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: tfm_g2n/train/f/d/b/fdb1435e4a.npy  
  inflating: tfm_g2n/train/f/d/b/fdb14b7377.npy  
  inflating: tfm_g2n/train/f/d/b/fdb168823b.npy  
  inflating: tfm_g2n/train/f/d/b/fdb1719211.npy  
  inflating: tfm_g2n/train/f/d/b/fdb1a48774.npy  
  inflating: tfm_g2n/train/f/d/b/fdb1d4f709.npy  
  inflating: tfm_g2n/train/f/d/b/fdb1dc62a0.npy  
  inflating: tfm_g2n/train/f/d/b/fdb1e85265.npy  
  inflating: tfm_g2n/train/f/d/b/fdb1f724dc.npy  
  inflating: tfm_g2n/train/f/d/b/fdb2661a8e.npy  
  inflating: tfm_g2n/train/f/d/b/fdb27e9e38.npy  
  inflating: tfm_g2n/train/f/d/b/fdb2817b6c.npy  
  inflating: tfm_g2n/train/f/d/b/fdb28781fb.npy  
  inflating: tfm_g2n/train/f/d/b/fdb28e9c17.npy  
  inflating: tfm_g2n/train/f/d/b/fdb2a10846.npy  
  inflating: tfm_g2n/train/f/d/b/fdb2a52f69.npy  
  inflating: tfm_g2n/train/f/d/b/fdb2a700e4.npy  
  inflating: tfm_g2n/train/f/d/b/fdb2ccc0a8.npy  
  inflating: tfm_g2n/train/f/d/b/fdb2e6

In [ ]:
from google.colab import drive

# Model

## Libraries

In [ ]:
import pandas as pd
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.metrics import AUC
import torch
from nnAudio.Spectrogram import CQT1992v2

/usr/local/lib/python3.7/dist-packages/nnAudio/Spectrogram.py:7: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  category=Warning,


## Setup variables

In [ ]:
train_labels = pd.read_csv("/content/tfm_g2n/training_labels.csv")
train_labels.head()

,id,target
0,00000e74ad,1
1,00001f4945,0
2,0000661522,0
3,00007a006a,0
4,0000a38978,1


In [ ]:
train_labels.shape

(560000, 2)

In [ ]:
# Assign the test IDs
sample_submission = pd.read_csv("/content/tfm_g2n/sample_submission.csv")
X_test = sample_submission[['id']]

In [ ]:
X = train_labels[['id']]
y = train_labels['target'].astype('int8').values

In [ ]:
#Como tenemos los datos de test en otro sitio, el split será de train y validation
X_train, x_valid, Y_train, y_valid = train_test_split(X, y, random_state = 42, stratify = y)
#his stratify parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to parameter stratify.

In [ ]:
# let's define some signal parameters
sample_rate = 2048 # data is provided at 2048 Hz
signal_length = 2 # each signal lasts 2 s
fmin, fmax = 20, 1024 # filter above 20 Hz, and max 1024 Hz (Nyquist freq = sample_rate/2)
hop_length = 64 # hop length parameter for the stft

In [ ]:
cq_transform = CQT1992v2(sr=sample_rate, fmin=fmin, fmax=fmax, hop_length=hop_length)
# CQT199v2 is used instead of GWpy to transform the wave data into CQT bc this performs the operation much faster and is GPU comp. Also is from PyTorch and thus takes torch.tensor as its inputs instead of tf.tensor

CQT kernels created, time used = 0.1030 seconds


/usr/local/lib/python3.7/dist-packages/nnAudio/utils.py:429: SyntaxWarning: If fmax is given, n_bins will be ignored
  warnings.warn("If fmax is given, n_bins will be ignored", SyntaxWarning)


#### Functions

In [ ]:
def get_npy_filepath(id_, is_train=True):
    if is_train:
        return f'/content/tfm_g2n/train/{id_[0]}/{id_[1]}/{id_[2]}/{id_}.npy'
    else:
        return f'/content/tfm_g2n/test/{id_[0]}/{id_[1]}/{id_[2]}/{id_}.npy'

In [ ]:
# function to load the file, preprocess, return the respective Constant Q-transform
# the Cqt function
# preprocess function
#Both in Pytorch and Tensorflow, the .numpy() method is pretty much straightforward. It converts a tensor object into an numpy.ndarray object. This implicitly means that the converted tensor will be now processed on the CPU.

def parse_function(id_path):
    # load the npy file
    signals = np.load(id_path.numpy())
    
    # loop through each signal #There are 3 signal as explained before for each interferometers
    for i in range(signals.shape[0]):
        # normalize the signal data
        signals[i] /= np.max(signals[i])
    
    # stack the arrays into a single vector #horizontal stack
    signals = np.hstack(signals)
    
    # convert the signals to torch.tensor to pass to CQT # tensor conversion
    signals = torch.from_numpy(signals).float()
    
    # get the CQT #getting the image from CQT transform
    image = cq_transform(signals)
    
    # conver the image from torch.tensor to array
    image = np.array(image)
    
    # transpose the image to get right orientation
    image = np.transpose(image,(1,2,0))
    
    # conver the image to tf.tensor and return
    return tf.convert_to_tensor(image)

In [ ]:
# the tf_function which is called in the data pipeline. This runs as TF function
def tf_parse_function(id_path, y=None):
    # pass the id_path to the py_function parse_function
    [x] = tf.py_function(func=parse_function, inp=[id_path], Tout=[tf.float32])
    
#     x.set_shape(signal_shape) # signal_shape
    x = tf.ensure_shape(x, input_shape)
    
    # if train/valid then return x, y; for test only return x
    if y is None:
        return x
    else:
        return x, y

In [ ]:
def get_shape(df):
  input_shape = parse_function(tf.convert_to_tensor(get_npy_filepath(df["id"].values[0])))
  return input_shape.shape


In [ ]:
def get_train(X_train=X_train, Y_train=Y_train):
  # Get the data filepaths as tensor_slices
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train["id"].apply(get_npy_filepath).values, Y_train))
  # shuffle the dataset
  train_dataset = train_dataset.shuffle(len(X_train))
  # Apply the map method to tf_parse_function()
  train_dataset = train_dataset.map(tf_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
  # set batcg size of the dataset
  train_dataset = train_dataset.batch(batch_size)
  # Prefetch the data
  train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
  return train_dataset

In [ ]:
def get_valid(x_valid,y_valid):
  # Get the data filepaths as tensor_slices
  valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid["id"].apply(get_npy_filepath).values, y_valid))
  # Apply the map method to tf_parse_function()
  valid_dataset = valid_dataset.map(tf_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
  # set batcg size of the dataset
  valid_dataset = valid_dataset.batch(batch_size)
  # Prefetch the data
  valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)
  return valid_dataset

In [ ]:
def get_test(X_test):
  # Get the data filepaths as tensor_slices
  test_dataset = tf.data.Dataset.from_tensor_slices((X_test["id"].apply(get_npy_filepath, is_train=False).values))
  # Apply the map method to tf_parse_function()
  test_dataset = test_dataset.map(tf_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
  # set batcg size of the dataset
  test_dataset = test_dataset.batch(batch_size)
  # Prefetch the data
  test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)
  return test_dataset

#### Define the dataset object

In [ ]:
# Input shape
input_shape = get_shape(train_data)

In [ ]:
train_dataset = get_train(X_train, Y_train)

In [ ]:
valid_dataset = get_valid(x_valid, y_valid)

In [ ]:
test_dataset = get_test(X_test)

# pruebas modelo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam

In [ ]:
def get_model(input_shape=input_shape):
    model = Sequential()
    model.add(Conv2D(filters=16,
                     input_shape=input_shape,
                     kernel_size=3,
                     activation="relu",
                     name="Conv_01"))
    model.add(BatchNormalization(name="Batch_01"))
    model.add(MaxPooling2D(pool_size=(2),name="Pool_01"))
    model.add(Conv2D(filters=32,
                     kernel_size=3,
                     dilation_rate=1,
                     activation="relu",
                     name="Conv_02"))
    model.add(BatchNormalization(name="Batch_02"))
    model.add(MaxPooling2D(pool_size=(2),name="Pool_03"))
    model.add(Conv2D(filters=64,
                     kernel_size=3,
                     dilation_rate=1,
                     activation="relu",
                     name="Conv_03"))
    model.add(BatchNormalization(name="Batch_03"))
    model.add(MaxPooling2D(pool_size=(2), name="Pool_03"))
    model.add(Flatten(name="Flatten"))
    model.add(Dense(units=512,
                    activation="relu"
                    name="Dense_01"))
    model.add(Dense(units=64,
                    activation="relu"
                    name="Dense_02"))
    model.add(Dense(1, activation='sigmoid', name="Output"))
    model.compile(optimizer = Adam(lr=1e-3),loss='binary_crossentropy',metrics=[[AUC(), 'accuracy']])
    return model


In [ ]:
model = get_model(input_shape)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 67, 191, 16)       160       
                                                                 
 batch_normalization_6 (Batc  (None, 67, 191, 16)      64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 33, 95, 16)       0         
 2D)                                                             
                                                                 
 activation_6 (Activation)   (None, 33, 95, 16)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 31, 93, 32)        4640      
                                                                 
 batch_normalization_7 (Batc  (None, 31, 93, 32)      

In [114]:
model.fit(train_dataset, validation_data=valid_dataset, epochs = 3)

Epoch 1/3
1680/1680 [==============================] - 2108s 1s/step - loss: 0.4560 - auc_1: 0.8446 - accuracy: 0.7743 - val_loss: 1.3536 - val_auc_1: 0.7994 - val_accuracy: 0.5164
Epoch 2/3
1680/1680 [==============================] - 2083s 1s/step - loss: 0.4506 - auc_1: 0.8479 - accuracy: 0.7772 - val_loss: 0.5882 - val_auc_1: 0.8236 - val_accuracy: 0.6963
Epoch 3/3
1680/1680 [==============================] - 2086s 1s/step - loss: 0.4453 - auc_1: 0.8515 - accuracy: 0.7808 - val_loss: 0.8200 - val_auc_1: 0.8206 - val_accuracy: 0.6133


In [117]:
model.fit(x=valid_dataset, epochs=3, verbose=1)

Epoch 1/3
560/560 [==============================] - 545s 972ms/step - loss: 0.4319 - auc_1: 0.8597 - accuracy: 0.7900
Epoch 2/3
560/560 [==============================] - 537s 958ms/step - loss: 0.4247 - auc_1: 0.8644 - accuracy: 0.7947
Epoch 3/3
560/560 [==============================] - 552s 986ms/step - loss: 0.4157 - auc_1: 0.8700 - accuracy: 0.8008


In [120]:
# save the model
model.save('/content/tfm_g2n/model_CNN.h5')

In [118]:
  # predict the test dataset using CNN
preds_n = model.predict(test_dataset)

In [119]:
sample_submission['target'] = preds_n

# Save the DataFrame to a file
sample_submission.to_csv("/content/tfm_g2n/submission_nueva", index=False)